## LSTM Regression model
1. read news dataset and caluculate sentiment of headline
2. compue the mean of sentiment score on news headline each day
3. concatenate the data set with stock dataset to get stcok value
4. normalize stocks value with MinMAxscaler
5. Fit the sentiment score and normalized stock price into the model
6. compare the model performance with different windows size


In [ ]:
import quandl
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

In [9]:
import quandl
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras import regularizers
from keras import optimizers
#

def stocks(start_date, end_date):
# use quandl to acquire nasdaq composite
    ndq = quandl.get("NASDAQOMX/COMP-NASDAQ",
                        start_date = start_date, 
                        end_date = end_date)
    ndq_df = ndq.reset_index()
    val = ndq_df['Index Value'].tolist()
    return ndq_df, val

def concat_headline(df):
    result_1 = df[['date', 'sentiment']]
    result_2 = result_1.groupby('date').cumcount() + 1
    result_3 = result_1.set_index(['date', result_2]).unstack().sort_index(1, level=1)
    result_3.columns = ['_'.join(map(str,i)) for i in result_3.columns]
    result_3 = result_3.reset_index()
    result_3['date'] = pd.to_datetime(result_3['date'])
    return result_3

def process(data):
    data['date'] = pd.to_datetime(data['date'])
    df = data.sort_values(by='date')
    df_1 = df.loc[(df['date'] >= '20140101') & (df['date'] <= '2019-04-04')]
    df_1['headline'] = df_1['headline'].fillna(" ")
    sid = SentimentIntensityAnalyzer()
    df_1['sentiment'] = df_1['headline'].apply(lambda x:sid.polarity_scores(x)['compound'] if (len(x)>3) else float(0))
    df_2 = concat_headline(df_1)
    return df_2

def senti_mean(df_2):
    # calculate sentiment mean of headline from news every day
    ttl=[]
    for row in range(df_2.shape[0]):
        lst=[]
        for col in range(1, df_2.shape[1]):
            if (df_2.iloc[row, col] is not None) or (pd.notnull(df_2.iloc[row, col])):
                    lst.append(df_2.iloc[row, col].astype(float))
        cleanedList = [x for x in lst if str(x) != 'nan']
        arr_mean = np.mean(cleanedList, axis=0)
        ttl.append(arr_mean)
    return ttl

def min_max(df_2):
    stocks_df1, val = stocks('2014-01-01', '2019-04-04')
    stocks_df2 = stocks_df1[['Trade Date', 'Index Value']]
    aa = stocks_df2.merge(df_2, how='left', left_on='Trade Date', right_on='date')
    
    # MinaxScaler
    stock_value = aa['Index Value'].values.astype('float32')
    Nas_stock_prices = stock_value.reshape(aa.shape[0], 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    Nas_stock_prices_1 = scaler.fit_transform(Nas_stock_prices)
    aa['min_max_stock'] = Nas_stock_prices_1
    aa['Label'] = aa['min_max_stock'].shift(-1)
    aa = aa.iloc[:-1, :]
    return aa, scaler

def processData(data,lb):
    # chunk data
    X,Y = [],[]
    for i in range(data.shape[0]-lb-1):
        X.append(data[i:(i+lb),:])
        Y.append(data[(i+lb),:])
    return np.array(X), np.array(Y)

def train_data(aa):
    ttl = []
    for row in range(aa.shape[0]):
        lst = []
        lst.append(float(aa.iloc[row,-3]))
        lst.append(float(aa.iloc[row,-2]))
        lst.append(float(aa.iloc[row,-1]))
        ttl.append(lst)
    return ttl

def lstm_model(x_train, y_train, window_size):
    
    filepath="model/model_15.hdf5"
    checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True)
    
    #Build the model
    model = Sequential()
    model.add(LSTM(256, input_shape=(window_size, x_train.shape[-1])))
    model.add(Dense(1))
    model.compile(optimizer='adam',loss='mse', metrics=['mse', 'mae', 'mape', 'cosine'])

    #Fit model with history to check for overfitting
    history = model.fit(x_train, y_train, epochs=500, batch_size=50
                        ,validation_split=0.1,verbose=0, shuffle=False, callbacks=[checkpoint])
    return model
if __name__ == "__main__":
    data = pd.read_pickle('../..Data/ProcessedNews.pkl')
    df_2 = process(data)
    ttl = senti_mean(df_2)
    df_2['average_senti'] = ttl
    aa, scaler = min_max(df_2)
    ttl = train_data(aa)
    arr_data = np.asarray(ttl)
    window_size = 15
    X,y = processData(arr_data, window_size)
    x_train,x_test = X[:int(X.shape[0]*0.80), :, :-1],X[int(X.shape[0]*0.80):, :, :-1]
    y_trai n,y_test = y[:int(y.shape[0]*0.80), -2],y[int(y.shape[0]*0.80):, -2]
    model = lstm_model(x_train, y_train, window_size)
    
#     Xt = model.predict(x_test)
#     print(scaler.inverse_transform(Xt))
    print(model.metrics_names)
    print(model.evaluate(x_test,y_test))
    
    

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['loss', 'mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'cosine_proximity']
262/262 [==============================] - 0s 201us/step
[0.0005391922518113301, 0.0005391922518113301, 0.01776900385570435, 2.210916217956834, -1.0]


In [2]:
# ['loss', 'mean_squared_error', 'mean_absolute_error']
#### window size 15
# 
#### window size 20
# [0.0013658519971659015, 0.0013658519971659015, 0.0329189677633545]
#### window size 30
# [0.0005190663024401976, 0.0005190663024401976, 0.016322801936000702]
#### window size 45
# [0.0013566281922976486, 0.0013566281922976486, 0.03261727234348655]
# window size 50
# [0.0007157920065390713, 0.0007157920065390713, 0.021968317645437576, 2.665913009176067, -1.0]